# Synthetic Population

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

In [ ]:
LONDON_BOUNDARY_FILE_URL = 'https://files.datapress.com/london/dataset/statistical-gis-boundary-files-london/2016-10-03T13:52:28/statistical-gis-boundaries-london.zip'
LONDON_CENSUS_LABOUR_URL = 'https://files.datapress.com/london/dataset/2011-census-labour-and-qualifications/visualisation-data-labour.zip'
LONDON_CENSUS_QUALIFICATION_URL = 'https://files.datapress.com/london/dataset/2011-census-labour-and-qualifications/visualisation-data-qualifications.zip'
LONDON_CENSUS_WARD_POPULATION_URL = 'https://files.datapress.com/london/dataset/2011-census-demography/ward-pop-ONS-GLA-Census.xls'
LONDON_CENSUS_BOROUGH_POPULATION_URL = 'https://files.datapress.com/london/dataset/2011-census-demography/london-unrounded-data.xls'
LONDON_CENSUS_HOUSEHOLD_URL = 'https://files.datapress.com/london/dataset/2011-census-households-families/visualisation-data-households.zip'
LONDON_CENSUS_HOUSING_URL = 'https://files.datapress.com/london/dataset/2011-census-housing/visualisation-data-housing.zip'

LABOUR_FILE_PATH = Path('./LABOUR.xlsx')
QUALIFICATION_FILE_PATH = Path('./QUALIFICATIONS.xlsx')
HOUSEHOLDS_FILE_PATH = Path('./HOUSEHOLDS.xlsx')
HOUSING_FILE_PATH = Path('./HOUSING.xlsx')
BUILD_FOLDER = Path('./build')
BUILD_FOLDER.mkdir(parents=True, exist_ok=True)

In [ ]:
NUMBER_USUAL_RESIDENTS = 254926 # http://www.haringey.gov.uk/sites/haringeygovuk/files/2011_census_haringey_briefing_on_key_statistics.pdf
NUMBER_HOUSEHOLDS = 101955

## Helper Functions

In [ ]:
import io
import zipfile
import tempfile
import requests
import requests_cache

requests_cache.install_cache((BUILD_FOLDER / 'cache').as_posix())

def read_census_file(url, filename):
    """Reads census 2011 data from the London data store.
    
    The dataset is reduced to Haringey and ward resolution. All other data is discarded.
    """
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    with tempfile.TemporaryDirectory(prefix='london-census-files') as tmpdir:
        z.extractall(path=tmpdir)
        path_to_temp_file = Path(tmpdir) / filename
        df = pd.read_excel(
            path_to_temp_file, 
            sheetname='2011 Data',
            skiprows=[0],
            header=[0]
        )
    df.rename(columns={'Unnamed: 1': 'area_type'}, inplace=True)
    df['area_type'] = df['area_type'].ffill()
    df = df[(df.DISTLABEL == 'Haringey') & (df.area_type == 'ward')]
    del df['DISTLABEL']
    del df['area_type']
    del df['ZONEID']
    del df['Unnamed: 2']
    df.set_index('ZONELABEL', inplace=True)
    df.index.rename('ward', inplace=True)
    return df

In [ ]:
def reduce_census_data_to_tottenham(census_data):
    return census_data[census_data.index.map(lambda label: 'Tottenham' in label)].sum()

In [ ]:
import random


def create_individual(df):
    random_number = random.uniform(0, 1)
    summed_probability = 0
    for i in df.index:
        if random_number < df.ix[i, 'total'] + summed_probability:
            return df.ix[i, df.columns[:-1]].values
        else:
            summed_probability += df.ix[i, 'total']
    raise ValueError('doh!')
    

def create_synthetic_population(df, column_names, pop_size, seed='syntheticpopulation'):
    norm_df = df.copy()
    norm_df.total = df.total / df.total.sum()
    return pd.DataFrame(
        data=(create_individual(norm_df) for i in range(pop_size)), 
        columns=column_names
    )

## Example not considering households

Two attributes with two categories each:

* age: 0-50yrs, 50-100yrs
* sex: m, f

In [ ]:
from enum import Enum

class OrderedEnum(Enum):
    def __ge__(self, other):
        if self.__class__ is other.__class__:
            return self.value >= other.value
        return NotImplemented
    def __gt__(self, other):
        if self.__class__ is other.__class__:
            return self.value > other.value
        return NotImplemented
    def __le__(self, other):
        if self.__class__ is other.__class__:
            return self.value <= other.value
        return NotImplemented
    def __lt__(self, other):
        if self.__class__ is other.__class__:
            return self.value < other.value
        return NotImplemented

class Age(OrderedEnum):
    AGE0_50 = 1
    AGE50_100 = 2
    
class Sex(OrderedEnum):
    MALE = 1
    FEMALE = 2
        

Let's create some microdata, the seed for the algorithm.

In [ ]:
p1 = (Age.AGE0_50, Sex.MALE)
p2 = (Age.AGE0_50, Sex.MALE)
p3 = (Age.AGE50_100, Sex.MALE)
p4 = (Age.AGE0_50, Sex.FEMALE)
p5 = (Age.AGE50_100, Sex.FEMALE)

Let's make up some statistics about the entire population.

In [ ]:
averages = {
    Age.AGE0_50: 75,
    Age.AGE50_100: 25,
    Sex.MALE: 65,
    Sex.FEMALE: 35
}

## Iterative Proportional Fitting

In [ ]:
df_in = pd.DataFrame(
        {
            'sex': [Sex.MALE, Sex.MALE, Sex.FEMALE, Sex.FEMALE],
            'age': [Age.AGE0_50, Age.AGE50_100, Age.AGE0_50, Age.AGE50_100],
            'total': [2, 1, 1, 1]
        }
    )
df_in

In [ ]:
xip = df_in.groupby('sex')['total'].sum()
xpj = df_in.groupby('age')['total'].sum()

xip.ix[Sex.MALE] = averages[Sex.MALE]
xip.ix[Sex.FEMALE] = averages[Sex.FEMALE]

xpj.ix[Age.AGE0_50] = averages[Age.AGE0_50]
xpj.ix[Age.AGE50_100] = averages[Age.AGE50_100]

aggregates = [xip, xpj]
dimensions = [['sex'], ['age']]

In [ ]:
from ipfn import *

IPF = ipfn.ipfn(
    df_in,
    aggregates, 
    dimensions
)
df_out = IPF.iteration()
df_out

In [ ]:
df_out.groupby('sex').sum()

In [ ]:
df_out.groupby('age').sum()

These numbers correctly mimic the population statistics.

Next, based on these numbers let's create a synthetic population. Assuming the result can be understood as a joint probability mass function, we can run 100 monte carlo draws to draw 100 individuals from this function.

In [ ]:
create_individual(df_out)

In [ ]:
synthetic_population = create_synthetic_population(
    df=df_out,
    column_names=['age', 'sex'],
    pop_size=100
)

In [ ]:
synthetic_population

In [ ]:
synthetic_population.describe()

That's the synthetic population!

These numbers diverge slightly from the given population statistics but that's due to the nondeterministic drawing.

In [ ]:
# TODO: consider zones
# TODO: consider households

## Statistical Data

### Read Usual Resident Data

In [ ]:
def read_ward_population_data(url):
    """Reads census 2011 demographic data on ward level from the London data store.
    
    The dataset is reduced to Haringey and ward resolution. All other data is discarded.
    """
    r = requests.get(url)
    df = pd.read_excel(
        io.BytesIO(r.content), 
        sheetname='2011 Census',
        skiprows=[0],
        header=[0]
    )
    df = df.ix[:, :23] # only totals, cut sex specifics
    df = df[df.Borough == 'Haringey']
    del df['Borough']
    del df['Persons: All Ages'] # cut totals
    del df['Ward Code']
    df.set_index('Ward Name', inplace=True)
    return df

In [ ]:
usual_residents = read_ward_population_data(LONDON_CENSUS_WARD_POPULATION_URL)
usual_residents.columns

In [ ]:
assert usual_residents.sum().sum() == NUMBER_USUAL_RESIDENTS

### Read total number of fifteen year old

The number of fifteen year old is important as it is used to divide adults from youth in the census (see lateron). The ward population data set cuts based on 5 years and hence does not cut between youth and adults. For reference, the total number of fifteen year old is read in here.

In [ ]:
def read_borough_population_data(url):
    """Reads census 2011 demographic data on borough level from the London data store."""
    r = requests.get(url)
    df = pd.read_excel(
        io.BytesIO(r.content), 
        sheetname='Persons',
        skiprows=[0],
        header=[0]
    )
    df.drop(df.columns[[0, 2]], axis=1, inplace=True)
    df.drop([0, 34, 35, 36, 37, 38], axis=0, inplace=True)
    df.rename(columns={'Unnamed: 1': 'ward'}, inplace=True)
    df.set_index('ward', inplace=True)
    df = df.astype(np.int16)
    return df

In [ ]:
total_fifteen_haringey = read_borough_population_data(LONDON_CENSUS_BOROUGH_POPULATION_URL).ix['Haringey', 15]
total_fifteen_haringey

### Read Economic Data

In [ ]:
qualification_data = read_census_file(LONDON_CENSUS_QUALIFICATION_URL, QUALIFICATION_FILE_PATH)

In [ ]:
qualification_data.columns

In [ ]:
assert qualification_data.sum().sum() == usual_residents.ix[:, '15 to 19':].sum().sum() - total_fifteen_haringey

Qualification data is available for every usual resident starting from age 16. Unfortunately there is no data available on how many residents are older or younger than 16 (only 15) in the ward population data set. But obviously, we can infer the number of residents below 16 from the qualification data.

In [ ]:
younger_than_sixteen = usual_residents.sum(axis=1) - qualification_data.sum(axis=1)
younger_than_sixteen.name = 'usual residents below 16'
younger_than_sixteen.sum()

In [ ]:
usual_residents_age_fifteen = younger_than_sixteen - usual_residents.ix[:, :'10 to 14'].sum(axis=1)
usual_residents_age_fifteen.name = 'usual residents age 16'
assert usual_residents_age_fifteen.sum() == total_fifteen_haringey

In [ ]:
qualification_data['below_16'] = younger_than_sixteen
assert qualification_data.sum().sum() == usual_residents.sum().sum()

In [ ]:
labour_data = read_census_file(LONDON_CENSUS_LABOUR_URL, LABOUR_FILE_PATH)
labour_data.drop(labour_data.columns[10:], axis=1, inplace=True)
labour_data.columns

In [ ]:
assert labour_data.sum().sum() == usual_residents.ix[:, '15 to 19':'70 to 74'].sum().sum() - total_fifteen_haringey

Labour data is available for every usual resident between age 16 and 74.

In [ ]:
labour_data['below_16'] = younger_than_sixteen
labour_data['above_74'] = usual_residents.ix[:, '75 to 79':].sum(axis=1)
assert labour_data.sum().sum() == usual_residents.sum().sum()

### Read Households

In [ ]:
household_data = read_census_file(LONDON_CENSUS_HOUSEHOLD_URL, HOUSEHOLDS_FILE_PATH)
housing_data = read_census_file(LONDON_CENSUS_HOUSING_URL, HOUSING_FILE_PATH)

In [ ]:
assert (housing_data['Unshared dwelling: Total'] + housing_data['Shared dwelling']).sum() == NUMBER_HOUSEHOLDS

In [ ]:
assert NUMBER_USUAL_RESIDENTS - younger_than_sixteen.sum() == \
    (household_data['Single (never married or never registered a same-sex civil partnership)'] + 
     household_data['Married'] +
     household_data['In a registered same-sex civil partnership'] +
     household_data['Separated (but still legally married or still legally in a same-sex civil partnership)'] + 
     household_data['Divorced or formerly in a same-sex civil partnership which is now legally dissolved'] + 
     household_data['Widowed or surviving partner from a same-sex civil partnership']).sum()

In [ ]:
household_data.drop(household_data.columns[:29], axis=1, inplace=True)
household_data.drop(household_data.columns[11:], axis=1, inplace=True)

In [ ]:
household_data.columns

[Definition of a dependent child](http://www.ons.gov.uk/ons/guide-method/census/2011/census-data/2011-census-data/2011-first-release/2011-census-definitions/2011-census-glossary.pdf) in the 2011 census:

> A dependent child is any person aged 0 to 15 in a household (whether or not in a family) or a person aged 16 to 18 in full-time education and living in a family with his or her parent(s) or grandparent(s). It does not include any people aged 16 to 18 who have a spouse, partner or child living in the household.

In [ ]:
assert NUMBER_HOUSEHOLDS == household_data.sum().sum()

In [ ]:
household_data['Couple with dependent children'] = (household_data['Married couple household: With dependent children'] + 
                                                    household_data['Same-sex civil partnership couple household: With dependent children'] + 
                                                    household_data['Cohabiting couple household: With dependent children'])
household_data['Lone parent with dependent children'] = household_data['Lone parent household: With dependent children']
household_data.drop(['Married couple household: With dependent children',
                     'Same-sex civil partnership couple household: With dependent children',
                     'Cohabiting couple household: With dependent children',
                     'Lone parent household: With dependent children'], axis=1, inplace=True)

In [ ]:
household_data['Couple without dependent children'] = (household_data['Married couple household: No dependent children'] + 
                                                       household_data['Same-sex civil partnership couple household: No dependent children'] + 
                                                       household_data['Cohabiting couple household: No dependent children'])
household_data['One person household'] += household_data['Lone parent household: No dependent children']
household_data.drop(['Married couple household: No dependent children',
                     'Same-sex civil partnership couple household: No dependent children',
                     'Cohabiting couple household: No dependent children',
                     'Lone parent household: No dependent children'], axis=1, inplace=True)

In [ ]:
household_data['Multi-person household'] = (household_data['Multi-person household: Other'] +
                                            household_data['Multi-person household: All full-time students'])
household_data.drop(['Multi-person household: All full-time students',
                     'Multi-person household: Other'], axis=1, inplace=True)

In [ ]:
household_data.sum()

In [ ]:
assert NUMBER_HOUSEHOLDS == household_data.sum().sum()

## Seed

As for the moment there is no micro census data available, we'll use the Time Use Survey data as seed.

In [ ]:
from pytus2000 import read_individual_file, individual

In [ ]:
DATA_FOLDER_PATH = Path('./data/UKDA-4504-tab/')
BUILD_FOLDER_PATH = Path('./build/')

In [ ]:
individual_cache = BUILD_FOLDER_PATH / 'pickled_individual_data'
try:
    individual_data = pd.read_pickle(individual_cache.as_posix())
except:
    individual_data = read_individual_file(DATA_FOLDER_PATH / 'tab' / 'individual_data_5.tab')
    individual_data.to_pickle(individual_cache.as_posix())

### Parameters of Individuals

In [ ]:
individual_data.groupby('ECONACT2').Q1A.count()

In [ ]:
econact2_to_census = {
    individual.ECONACT2.ECON_ACTIVE___EMPLOYEE___FULL_TIME: 'Economically active: Employee: Full-time',
    individual.ECONACT2.ECON_INACTIVE___LONG_TERM_SICK_DISABLED: 'Economically inactive: Long-term sick or disabled',
    individual.ECONACT2.ECON_INACTIVE___OTHER_REASONS_EG_TEMP_SICK__BELIEVES_NO_JOBS: 'Economically inactive: Other',
    individual.ECONACT2.ECON_INACTIVE___DK_REASONS: np.nan, #FIXME
    individual.ECONACT2.ADULT___NOT_CLASSIFIABLE_EITHER_EMP__UNEMP_OR_INACTIVE: np.nan,
    individual.ECONACT2.UNDER_16YRS___INELIGIBLE_FOR_EMPLOYMENT_QUESTIONS: 'below_16',
    individual.ECONACT2.ECON_ACTIVE___EMPLOYEE___PART_TIME: 'Economically active: Employee: Part-time',
    individual.ECONACT2.ECON_ACTIVE___SELF_EMPLOYED___FULL_TIME: 'Economically active: Self-employed',
    individual.ECONACT2.ECON_ACTIVE___SELF_EMPLOYED___PART_TIME: 'Economically active: Self-employed',
    individual.ECONACT2.ECON_ACTIVE___DK_EMPSELFFULLPART: np.nan, #FIXME
    individual.ECONACT2.ECON_ACTIVE___UNEMPLOYED_ILO_DEFINITION: 'Economically active: Unemployed',
    individual.ECONACT2.ECON_INACTIVE___RETIRED: 'Economically inactive: Retired',
    individual.ECONACT2.ECON_INACTIVE___FULL_TIME_STUDENT: 'Economically inactive: Student (including full-time students)',
    individual.ECONACT2.ECON_INACTIVE___LOOKING_AFTER_FAMILY_HOME: 'Economically inactive: Looking after home or family'
}

In [ ]:
individual_data.groupby('HIQUAL4').Q1A.count()

In [ ]:
hiqual4_to_census = { # FIXME all must be checked again
    individual.HIQUAL4.DEGREE_LEVEL_QUALIFICATION_OR_ABOVE: 'Level 4/5',
    individual.HIQUAL4.QUALIFICATIONS___CITY_AND_GUILDS___DK_LEVEL: np.nan,
    individual.HIQUAL4.QUALIFICATIONS___OTHER___BUT_DK_GRADELEVEL: np.nan,
    individual.HIQUAL4.NO_QUALIFICATIONS: 'No qualifications',
    individual.HIQUAL4.ELIGIBLE___NO_ANSWER: np.nan,
    individual.HIQUAL4.UNDER_16YRS___INELIGIBLE_FOR_QUALIFICATIONS_QUESTIONS: 'below_16',
    individual.HIQUAL4.HIGHER_EDN_BELOW_DEGREE_LEVEL_EG_HNC__NURSING_QUAL: 'Level 3', # FIXME correct?
    individual.HIQUAL4.A_LEVELS__VOCATIONAL_LEVEL_3_AND_EQUIVLNT_EG_AS_LEVEL__NVQ_3: 'Level 3',
    individual.HIQUAL4.O_LEVELS__GCSE_GRADE_A_C__VOCATIONAL_LEVEL_2_AND_EQUIVLNT: 'Level 2',
    individual.HIQUAL4.GCSE_BELOW_GRADE_C__CSE__VOCATIONAL_LEVEL_1_AND_EQUIVLNT: 'Level 1',
    individual.HIQUAL4.QUALIFICATION_BELOW_GCSEO_LEVEL_EG_TRADE_APPRENTICESHIPS: 'Apprenticeship',
    individual.HIQUAL4.OTHER_QUALIFICATION_INCL_PROFESSIONAL__VOCATIONAL__FOREIGN: 'Other qualificatinon',
    individual.HIQUAL4.QUALIFICATIONS___BUT_DK_WHICH: 'Other qualificatinon',# FIXME correct?
    individual.HIQUAL4.QUALIFICATIONS___GCSE___BUT_DK_GRADE: np.nan # FIXME correct?
}

### Parameters of households

In [ ]:
individual_data.groupby('HHTYPE4').IAGE.count()

In [ ]:
hhtype_to_census = {
    individual.HHTYPE4.SINGLE_PERSON_HOUSEHOLD: 'One person household',
    individual.HHTYPE4.SINGLE_PARENT___WITH_CHILDREN_GREATEREQUAL_16: 'Lone parent with dependent children',
    individual.HHTYPE4.TWO_OR_MORE_COUPLES_MARRIED_OR_COHAB_WITHWITHOUT_CHILDRN: 'Multi-person household',
    individual.HHTYPE4.SAME_SEX_COUPLES___SPONTANEOUSLY_DESCRIBED: 'Couple without dependent children',
    individual.HHTYPE4.UNCLASSIFIED___MARRIED_COUPLES_IN_COMPLEX_HHLDS: 'Multi-person household',
    individual.HHTYPE4.UNCLASSIFIED___COHABITING_COUPLES_IN_COMPLEX_HHLDS: 'Multi-person household',
    individual.HHTYPE4.UNCLASSIFIED___SINGLE_PARENTS_IN_COMPLEX_HHLDS: 'Multi-person household',
    individual.HHTYPE4.UNCLASSIFIED___OTHER_HHLDS_WITHOUT_COUPLES_EG_BROTHERS_DK: 'Multi-person household',
    individual.HHTYPE4.HHLDS_WITH_2_OR_MORE_UNRELATED_PEOPLE_ONLY: 'Multi-person household',
    individual.HHTYPE4.MARRIED_COUPLE___NO_CHILDREN_COUPLE_ONLY: 'Couple without dependent children',
    individual.HHTYPE4.MARRIED_COUPLE___WITH_CHILDREN_SMALLEREQUAL_15: 'Couple with dependent children',
    individual.HHTYPE4.MARRIED_COUPLE___WITH_CHILDREN_GREATEREQUAL_16: 'Couple with dependent children',
    individual.HHTYPE4.COHAB_COUPLE___NO_CHILDREN_COUPLE_ONLY: 'Couple without dependent children',
    individual.HHTYPE4.COHAB_COUPLE___WITH_CHILDREN_SMALLEREQUAL_15: 'Couple with dependent children',
    individual.HHTYPE4.COHAB_COUPLE___WITH_CHILDREN_GREATEREQUAL_16: 'Couple with dependent children',
    individual.HHTYPE4.SINGLE_PARENT____WITH_CHILDREN_SMALLEREQUAL_15: 'Lone parent with dependent children'
}

In [ ]:
## TODO filter city population

### Build Seed

In [ ]:
seed = pd.DataFrame(index=individual_data.index, columns=['labour', 'qualification', 'age', 'hhtype'])
seed.labour = individual_data.ECONACT2.map(econact2_to_census)
seed.labour[individual_data.IAGE > 74] = 'above_74'
seed.qualification = individual_data.HIQUAL4.map(hiqual4_to_census)
seed['age'] = individual_data.IAGE.copy()
seed['hhtype'] = individual_data.HHTYPE4.map(hhtype_to_census)
seed.head()

### Economically active and full time students

The statistics have the category of people that are full time students but still economically active. The TUS data does not have that category. The category we are looking for is the category of part time economically active and full time student.

In [ ]:
individual_data[(individual_data.ECONACT2 == individual.ECONACT2.ECON_INACTIVE___FULL_TIME_STUDENT) & 
                (individual_data.AGELEFT == individual.AGELEFT.NEVER_IN_FULL_TIME_EDUCATION)].IAGE.count()

Unfortunately, the categorization into labour status seems rather weak. For example, there is are 8 individuals that are categorized as a full time student that answered he/she has never been in full time education.

In [ ]:
part_time_working_students = individual_data[
    (individual_data.ECONACT2 == individual.ECONACT2.ECON_ACTIVE___EMPLOYEE___PART_TIME) &
    (individual_data.AGELEFT == individual.AGELEFT.STILL_IN_EDUCATION)
]

In [ ]:
part_time_working_students.IAGE.count()

In [ ]:
for index, student in part_time_working_students.iterrows():
    seed.ix[index, 'labour'] = 'Economically active: Full-time student'

### Prepare Index for HIPF

In [ ]:
sn1_plus_sn2 = seed.index.droplevel(2)
seed['household_id'] = list(sn1_plus_sn2)
seed.reset_index(inplace=True)
seed.rename(columns={'SN3': 'person_id'}, inplace=True)
seed.set_index(['household_id', 'person_id'], inplace=True)
seed.drop(['SN1', 'SN2'], axis=1, inplace=True)
seed.head()

### Handle NaN and invalid households

A household is invalid if the amount of individuals we have do not match the household type. For example, a couple without children household must have exactly two individuals

In [ ]:
seed.dropna(axis='index', how='any', inplace=True)
assert not seed.isnull().any().any()

In [ ]:
household_types = seed.groupby(seed.index.get_level_values(0)).hhtype.first()
household_sizes = seed.groupby(seed.index.get_level_values(0)).hhtype.count()
mask_couples_children = household_sizes[(household_types == 'Couple with dependent children') & (household_sizes <= 2)]
mask_couples_no_children = household_sizes[(household_types == 'Couple without dependent children') & (household_sizes != 2)]
invalids = (
    household_sizes[(household_types == 'Couple with dependent children') & (household_sizes <= 2)] |
    household_sizes[(household_types == 'Couple without dependent children') & (household_sizes != 2)] |
    household_sizes[(household_types == 'Lone parent with dependent children') & (household_sizes < 2)] |
    household_sizes[(household_types == 'Multi-person household') & (household_sizes <= 2)]
)

seed.drop(labels=invalids.index, level=0, inplace=True)


print("{} households are invalid and were removed.".format(invalids.count()))

In [ ]:
household_types = seed.groupby(seed.index.get_level_values(0)).hhtype.first()
household_sizes = seed.groupby(seed.index.get_level_values(0)).hhtype.count()
assert (household_sizes[household_types == 'One person household'] == 1).all()
assert (household_sizes[household_types == 'Couple with dependent children'] > 2).all()
assert (household_sizes[household_types == 'Couple without dependent children'] == 2).all()
assert (household_sizes[household_types == 'Lone parent with dependent children'] > 1).all()
assert (household_sizes[household_types == 'Multi-person household'] > 2).all()

### Tests

In [ ]:
assert not ((seed.labour == 'above_74') & (seed.age <= 74)).any()
assert not ((seed.labour == 'below_16') & (seed.age >= 16)).any()
assert not ((seed.labour == 'below_16') & (seed.qualification != 'below_16')).any()
assert not ((seed.labour != 'below_16') & (seed.qualification == 'below_16')).any()

household_types = seed.groupby(seed.index.get_level_values(0)).hhtype.first()
household_sizes = seed.groupby(seed.index.get_level_values(0)).hhtype.count()
assert (household_sizes[household_types == 'One person household'] == 1).all()
assert (household_sizes[household_types == 'Couple with dependent children'] > 2).all()
assert (household_sizes[household_types == 'Lone parent with dependent children'] > 1).all()
assert (household_sizes[household_types == 'Couple without dependent children'] == 2).all()
assert (household_sizes[household_types == 'Multi-person household'] > 2).all()

In [ ]:
seed.count()

## Synthetic Population

### Using plain IPF

This performs a fit of the individuals only.

In [ ]:
df_in = seed.groupby(['labour', 'qualification']).count().reset_index().rename(columns={'age': 'total'})
assert df_in.total.sum() == seed.age.count()
df_in.head()

In [ ]:
xip = pd.Series(index=labour_data.columns)
xpj = pd.Series(index=qualification_data.columns)

for labour_status in labour_data.columns:
    xip[labour_status] = labour_data[labour_status].sum()

for qualification in qualification_data.columns:
    xpj[qualification] = qualification_data[qualification].sum()

aggregates = [xip, xpj]
dimensions = [['labour'], ['qualification']]

In [ ]:
from ipfn import *

IPF = ipfn.ipfn(
    df_in,
    aggregates, 
    dimensions
)
df_out = IPF.iteration()

In [ ]:
assert df_out.total.sum() - usual_residents.sum().sum() < 1
assert all((df_out.groupby('qualification').total.sum() - qualification_data.sum()) < 1)
assert all((df_out.groupby('labour').total.sum() - labour_data.sum()) < 1)

In [ ]:
df_out.head()

In [ ]:
#synthetic_population = create_synthetic_population(
#    df=df_out,
#    column_names=['qualification', 'labour'],
#    pop_size=usual_residents.sum().sum(),
#    seed='syntheticpopulation-haringey'
#)
#synthetic_population.describe()

### Using HIPF

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
from ktp.synthpop import fit_hipf

In [ ]:
weights = fit_hipf(
    reference_sample=seed,
    controls_households={'hhtype': household_data.sum().to_dict()},
    controls_individuals={
        'labour': labour_data.sum().to_dict(),
        'qualification': qualification_data.sum().to_dict()
    },
    residuals_tol=0.0001,
    weights_tol=0.0001,
    maxiter=100
)

In [ ]:
weights.describe()

In [ ]:
assert NUMBER_HOUSEHOLDS - weights.sum() < 0.1